# Multi-Cycle and Parameters

In this notebook, we cover the ability to simulation over a larger range of conditions inclusive of multi-cycling via the 'experimental' function call. First, we install and import PyBaMM.

In [ ]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import matplotlib.pyplot as plt

An experimental defines conditional progression of the battery, and is covers common protocols like a CC-CV operation. In the following definition, a cycle is defined by a tuple of operating instructions. In this case, the experiment consists of a cycle of constant current C/10 discharge, a one hour rest, a constant current (1 A) constant voltage (4.2 V) and another one hour rest, all of it repeated three times (notice the * 3).

In [ ]:
experiment = pybamm.Experiment(
    [
        ("Discharge at C/10 for 10 hours or until 3.0 V",
        "Rest for 1 hour",
        "Charge at 1 A until 4.1 V",
        "Hold at 4.1 V until 50 mA",
        "Rest for 1 hour"),
    ] * 3
)


Next, we re-introduce the DFN model,

In [ ]:
model = pybamm.lithium_ion.DFN()

and create our simulation, passing our experiment using a keyword argument

In [ ]:
sim = pybamm.Simulation(model, experiment=experiment)

We then solve and plot the solution

In [ ]:
sim.solve()
sim.plot()

As we have seen, experiments allow us to define complex simulations using a very simple syntax. The instructions can be of the form "(Dis)charge at x A/C/W", "Rest", or "Hold at x V". The running time should be a time in seconds, minutes or hours, e.g. "10 seconds", "3 minutes" or "1 hour". The stopping conditions should be a circuit state, e.g. "1 A", "C/50" or "3 V". 

Some examples of experiment instructions are:
```python
    "Discharge at 1C for 0.5 hours",
    "Discharge at C/20 for 0.5 hours",
    "Charge at 0.5 C for 45 minutes",
    "Discharge at 1 A for 90 seconds",
    "Charge at 200mA for 45 minutes (1 minute period)",
    "Discharge at 1 W for 0.5 hours",
    "Charge at 200 mW for 45 minutes",
    "Rest for 10 minutes (5 minute period)",
    "Hold at 1 V for 20 seconds",
    "Charge at 1 C until 4.1V",
    "Hold at 4.1 V until 50 mA",
    "Hold at 3V until C/50",
```

Optionally, each instruction can contain at the end the expression "(x minute period)" in which the period at which to record the simulation outputs during that instruction. To change the period for the whole experiment we can pass it as a keyword argument in the experiment.

Additionally, we can use the operators `+` and `*` on lists in order to combine and repeat cycles:

In [ ]:
[("Discharge at 1C for 0.5 hours", "Discharge at C/20 for 0.5 hours")] * 3 + [("Charge at 0.5 C for 45 minutes",)]

# Modifying parameter values

Previously, we've ran all of our models with the default parameter values; However, PyBaMM also allows you to tweak these settings for your application. In this tutorial, we will see how to change the parameters in PyBaMM. They can be viewed as,

In [ ]:
model.default_parameter_values

Note, the default parameter set is "Marquis2019".

## Changing the parameter set

PyBaMM has a number of in-built parameter sets (check the list [here](https://pybamm.readthedocs.io/en/latest/source/api/parameters/parameter_sets.html)), which can be selected via,

In [ ]:
parameter_values = pybamm.ParameterValues("Chen2020")

Likewise to above, we can display the parameters and corresponding values stored in the dictionary,

In [ ]:
parameter_values

or we can search for a particular parameter

In [ ]:
parameter_values.search("electrolyte")

To run a simulation with this parameter set, we can proceed as usual but passing the parameters as a keyword argument

In [ ]:
sim = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment)
sim.solve()
sim.plot()

More details on each subset can be found [here](https://github.com/pybamm-team/PyBaMM/tree/develop/pybamm/input/parameters).

## Change individual parameters

We often want to quickly change a small number of parameter values to investigate how the behaviour or the battery changes. In such cases, we can change parameter values without having to leave the notebook or script you are working in.

We will modify the contact resistance variable to 10 mOhm,

In [ ]:
parameter_values["Contact resistance [Ohm]"] = 0.01

Now we just need to run the simulation with the new parameter values

In [ ]:
sim = pybamm.Simulation(model, experiment=experiment, parameter_values=parameter_values)
sim.solve()
sim.plot()

# Simulating a  Drive-Cycle

We start by importing the drive-cycle in csv format,

In [ ]:
%%bash 

curl -o WLTP_M50_M3.csv https://raw.githubusercontent.com/BradyPlanden/IntelLiGent-Modelling-Workshop/main/Day1/notebooks/Data/WLTP_M50_M3.csv

You can then implement drive cycles importing the dataset and creating an interpolant to pass as the current function.

In [ ]:
import pandas as pd    # needed to read the csv data file
import numpy as np

drive_cycle_power = pd.read_csv("/content/WLTP_M50_M3.csv", comment="#", header=None).to_numpy()

We can then include the previous experiment definition as,

In [ ]:
# Create interpolant
timescale = parameter_values.evaluate(model.timescale)
power_interpolant = pybamm.Interpolant(drive_cycle_power[:, 0], drive_cycle_power[:, 1], timescale * pybamm.t)

# Set drive cycle power function
parameter_values.update({"Power function [W]": power_interpolant},check_already_exists=False)
t_eval = np.linspace(0, 1800, 7201)

Note that your drive cycle data can be stored anywhere, you just need to pass the path of the file. Then, again, the model can be solved as usual but notice that now, if `t_eval` is not specified, the solver will take the time points from the data set.

In [ ]:
model = pybamm.lithium_ion.SPMe({"operating mode": "power"})
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve(t_eval)
sim.plot(["Current [A]", "Terminal voltage [V]"])

Now, we can set the initial SOC for the drive-cycle by passing it via keyword arguments, 

In [ ]:
sim.solve(t_eval,initial_soc=0.717)
sim.plot(["Current [A]", "Terminal voltage [V]"])

Reusing the model comparison from the previous notebook, we can create a list for comparing models on the WLTP drive-cycle as:

In [ ]:
models = [
    pybamm.lithium_ion.SPM({"operating mode": "power"}),
    pybamm.lithium_ion.SPMe({"operating mode": "power"}),
    pybamm.lithium_ion.DFN({"operating mode": "power"}),
]

Note that we update the model definition for the power operation mode. Next, we loop through the formed list and append the solutions to the `sim` list.

In [ ]:
sims = []
for model in models:
    sim = pybamm.Simulation(model, parameter_values=parameter_values)
    sim.solve(t_eval)
    sims.append(sim)

Plotting the comparison as,

In [ ]:
pybamm.dynamic_plot(sims, time_unit="seconds")

# Accessing and saving simulation outputs

We can now access the solved variables directly to visualise or create our own plots. We first extract the solution object:

In [ ]:
solution = sims[0].solution

and now we can create a post-processed variable,

In [ ]:
t = solution["Time [s]"]
V = solution["Terminal voltage [V]"]

One option is to visualise the data set returned by the solver directly

In [ ]:
V.entries

which correspond to the data at the times

In [ ]:
t.entries

In addition, post-processed variables can be called at any time (by interpolation)

In [ ]:
V([200, 400, 780, 1236])  # times in seconds

## Saving the simulation and output data

In some cases simulations might take a long time to run so it is advisable to save in your computer so it can be analysed later without re-running the simulation. You can save the whole simulation doing:

In [ ]:
sims[1].save("SPMe.pkl")

If you now check the root directory of your notebooks you will notice that a new file called `"SPMe.pkl"` has appeared. We can load the stored simulation doing

In [ ]:
sim2 = pybamm.load("SPMe.pkl")

which allows the same manipulation as the original simulation would allow

In [ ]:
sim2.plot()

Alternatively, we can just save the solution of the simulation in a similar way

In [ ]:
sol = sim.solution
sol.save("SPMe_sol.pkl")

and load it in a similar way too

In [ ]:
sol2 = pybamm.load("SPMe_sol.pkl")
pybamm.dynamic_plot(sol2)

Another option is to just save the data for some variables

In [ ]:
sol.save_data("sol_data.pkl", ["Current [A]", "Terminal voltage [V]"])

or save in csv or mat format

In [ ]:
sol.save_data("sol_data.csv", ["Current [A]", "Terminal voltage [V]"], to_format="csv")
# matlab needs names without spaces
sol.save_data("sol_data.mat", ["Current [A]", "Terminal voltage [V]"], to_format="matlab",
              short_names={"Current [A]": "I", "Terminal voltage [V]": "V"})

If you are running on your local, removing the generated files can be completed through the below code block.

In [ ]:
import os
os.remove("SPMe.pkl")
os.remove("SPMe_sol.pkl")
os.remove("sol_data.pkl")
os.remove("sol_data.csv")
os.remove("sol_data.mat")

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()